In [33]:
# AI Model Implementation in Jupyter Notebook

## Cell 1: Import necessary libraries
import numpy as np
import pandas as pd
import sqlite3
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error


In [34]:
## Cell 2: Correctly connect to SQLite database and load data
# Absolute path to your SQLite database
db_path = r"C:\Users\HP\Documents\GitHub\vroomble\Vroomble Dataset\prediction_database.db"

# Connect explicitly
conn = sqlite3.connect(db_path)

# Load data from SQLite tables into DataFrames
car_model_data = pd.read_sql_query("SELECT * FROM Car_Model", conn)
car_modifications_data = pd.read_sql_query("SELECT * FROM Car_Modifications", conn)

# Verify loaded data
print("Car Model Data:")
display(car_model_data.head())

print("\nCar Modifications Data:")
display(car_modifications_data.head())



Car Model Data:


,Make,Model,Month,Base_Price_PHP,Monthly_Inflation_Rate
0,Honda,CR-V,2024-03,2200000,0.05
1,Honda,Civic,2024-03,1600000,0.05
2,Ford,Everest,2024-03,2100000,0.05
3,Toyota,Fortuner,2024-03,1900000,0.05
4,Mitsubishi,Mirage,2024-03,800000,0.05



Car Modifications Data:


,Make,Model,Month,Modification_Type,Car_Part,Modification_Cost_PHP,Monthly_Inflation_Rate
0,Honda,CR-V,2023-01,Street/Casual,Honda CR-V Chrome Door Handle Cover,5000,0.05
1,Honda,CR-V,2023-01,Offroad,Honda CR-V Heavy-Duty Skid Plate,15000,0.05
2,Honda,CR-V,2023-01,Offroad,Honda CR-V Off-Road Fender Flares,12000,0.05
3,Honda,CR-V,2023-01,Offroad,Honda CR-V Snorkel Air Intake System,18000,0.05
4,Honda,CR-V,2023-01,Offroad,Honda CR-V Suspension Lift Kit,20000,0.05


In [ ]:
# Cell 3: Prepare training data for the AI model

X = []  # Features: Base price, Modification Cost, Inflation rate, Months
y = []  # Target: Predicted Price

# Iterate through each row to prepare the dataset
for _, row in car_model_data.iterrows():
    base_price = row["Base_Price_PHP"]
    inflation_rate = row["Monthly_Inflation_Rate"]
    
    # Compute total modification cost for the model
    modifications = car_modifications_data[
        (car_modifications_data["Model"] == row["Model"])
    ]["Modification_Cost_PHP"].sum()

    # Train for different months (1 to 24 months)
    for months in range(1, 25):  # Training over 1 to 24 months
        predicted_price = (base_price + modifications) * ((1 + inflation_rate) ** (months / 12))

        # Store feature values including "months"
        X.append([base_price, modifications, inflation_rate, months])
        y.append(predicted_price)

# Convert lists to NumPy arrayss
X = np.array(X)
y = np.array(y)

In [36]:
# Cell 4: Scale features, train Linear Regression model, and save scaler/model
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train Linear Regression model
model = LinearRegression()
model.fit(X_scaled, y)

# Output trained parameters
print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)

# Save the scaler and trained model explicitly
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(model, 'trained_model.pkl')

print("Scaler and trained model saved successfully.")

Intercept: 3121883.811933096
Coefficients: [ 522947.67284188 1949883.51195421       0.           87850.05485555]
Scaler and trained model saved successfully.


In [37]:
##Cell 5: Accuracy Test
# Split data into training and testing sets (20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train the model using training data
model.fit(X_train, y_train)

# Make predictions on test data
y_pred = model.predict(X_test)

# Calculate model performance metrics
r2 = r2_score(y_test, y_pred)  # R-squared score
mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error

# Display accuracy metrics
print(f"R² Score (Accuracy): {r2:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")

R² Score (Accuracy): 0.9992
Mean Squared Error (MSE): 3732228349.1248


In [38]:
## Cell 5: Prediction function using the trained model
def predict_price(make, model_name, modification_type, selected_parts, months=12):
    """Predict car price based on modification and inflation rate."""
    # Fetch the base price and inflation rate
    query_model = """
        SELECT Base_Price_PHP, Monthly_Inflation_Rate FROM Car_Model
        WHERE Make = ? AND Model = ?
    """
    car_info = pd.read_sql_query(query_model, conn, params=(make, model_name))

    if car_info.empty:
        print(f"Error: '{make} {model_name}' not found in database.")
        return None

    base_price = car_info.iloc[0]["Base_Price_PHP"]
    inflation_rate = car_info.iloc[0]["Monthly_Inflation_Rate"]

    # Compute modification cost
    modification_cost = 0.0
    if selected_parts:
        placeholders = ','.join(['?'] * len(selected_parts))
        query_mod = f"""
            SELECT SUM(Modification_Cost_PHP) as total_mod_cost FROM Car_Modifications
            WHERE Modification_Type = ? AND Model = ? AND Car_Part IN ({placeholders})
        """
        params = [modification_type, model_name] + selected_parts
        modification_cost_df = pd.read_sql_query(query_mod, conn, params=params)

        modification_cost = modification_cost_df.iloc[0]['total_mod_cost']

        # Handle if no modifications found
        if modification_cost is None:
            print("Warning: No matching modifications found. Defaulting modification cost to 0.")
            modification_cost = 0

    # Fix: Ensure "months" is included in the model input
    future_inflation = (1 + inflation_rate) ** (months / 12)

    # Load scaler and trained model
    scaler = joblib.load('scaler.pkl')
    model = joblib.load('trained_model.pkl')

    # Fix: Now includes "months" when scaling features
    features_scaled = scaler.transform([[base_price, modification_cost, inflation_rate, months]])

    # Predict price
    predicted_price = model.predict(features_scaled)[0]

    # Display prediction result
    result = {
        "Make": make,
        "Model": model_name,
        "Modification Type": modification_type,
        "Selected Car Parts": selected_parts,
        "Base Price (PHP)": base_price,
        "Car Parts Cost (PHP)": modification_cost,
        "Current Total Price (PHP)": base_price + modification_cost,
        f"Predicted Price After {months} Months (PHP)": predicted_price
    }

    for key, value in result.items():
        print(f"{key}: {value}")

    return result



In [39]:
## Cell 6: Function to predict car price based on selected parts
# Example Test
test_result = predict_price(
    make='Toyota',
    model_name='Vios',
    modification_type='Street/Casual',
    selected_parts=['Toyota Vios Trunk Lip Spoiler', 'Toyota Vios Chrome Door Handle Cover'],
    months=12  # Now correctly passed to the function
)

print("\nPredicted Price After 12 Months:", test_result)




Make: Toyota
Model: Vios
Modification Type: Street/Casual
Selected Car Parts: ['Toyota Vios Trunk Lip Spoiler', 'Toyota Vios Chrome Door Handle Cover']
Base Price (PHP): 750000.0
Car Parts Cost (PHP): 276000
Current Total Price (PHP): 1026000.0
Predicted Price After 12 Months (PHP): 1073574.3459716295

Predicted Price After 12 Months: {'Make': 'Toyota', 'Model': 'Vios', 'Modification Type': 'Street/Casual', 'Selected Car Parts': ['Toyota Vios Trunk Lip Spoiler', 'Toyota Vios Chrome Door Handle Cover'], 'Base Price (PHP)': np.float64(750000.0), 'Car Parts Cost (PHP)': np.int64(276000), 'Current Total Price (PHP)': np.float64(1026000.0), 'Predicted Price After 12 Months (PHP)': np.float64(1073574.3459716295)}


In [40]:
conn.close()